# CSV比較ツール - S3連携版
## S3上の2つのPrefix間でCSVファイルを比較

### 1. ライブラリのインポート

In [ ]:
import pandas as pd
import boto3
from datetime import datetime
import io
from tqdm import tqdm
import gc
from botocore.exceptions import ClientError

In [ ]:
### 2. 設定

In [ ]:
# S3設定
S3_BUCKET = 'k-ishibashi-test'  # バケット名を設定
S3_PREFIX_A = 'data/before/'    # 比較元Prefix
S3_PREFIX_B = 'data/after/'     # 比較先Prefix
S3_OUTPUT_PREFIX = 'output/'    # 出力先Prefix
AWS_PROFILE = None              # AWSプロファイル（Noneでデフォルト）

# 処理設定
CHUNK_SIZE = 10000              # チャンクサイズ（行数）
MEMORY_THRESHOLD_MB = 100       # メモリ閾値（MB）

# デフォルトキー列・無視列
DEFAULT_KEY_COLUMNS = ['vin']
DEFAULT_IGNORE_COLUMNS = []

# ファイル別設定
FILE_SETTINGS = {
    'tccontract_20251217.csv': {
        'key_columns': ['vin'],
        'ignore_columns': []
    },
    'sample.csv': {
        'key_columns': ['vin'],
        'ignore_columns': []
    }
}

print(f'S3バケット: {S3_BUCKET}')
print(f'PrefixA: {S3_PREFIX_A}')
print(f'PrefixB: {S3_PREFIX_B}')
print(f'チャンクサイズ: {CHUNK_SIZE:,}行')
print(f'メモリ閾値: {MEMORY_THRESHOLD_MB}MB')

### 3. S3操作関数

In [25]:
def get_s3_client(profile=None):
    if profile:
        session = boto3.Session(profile_name=profile)
        return session.client('s3')
    return boto3.client('s3')

def list_s3_csv_files(s3_client, bucket, prefix):
    """S3バケット内のCSVファイル一覧を取得"""
    try:
        response = s3_client.list_objects_v2(Bucket=bucket, Prefix=prefix)
        if 'Contents' not in response:
            return []
        
        files = []
        for obj in response['Contents']:
            key = obj['Key']
            if key.endswith('.csv') and key != prefix:
                filename = key.split('/')[-1]
                files.append(filename)
        return files
    except ClientError as e:
        print(f"S3エラー: {e}")
        return []

def get_s3_file_size(s3_client, bucket, key):
    """S3ファイルサイズを取得（MB）"""
    try:
        response = s3_client.head_object(Bucket=bucket, Key=key)
        return response['ContentLength'] / (1024 * 1024)
    except ClientError:
        return 0

def read_csv_from_s3(s3_client, bucket, key):
    """S3からCSVを読み込み"""
    try:
        response = s3_client.get_object(Bucket=bucket, Key=key)
        return pd.read_csv(io.BytesIO(response['Body'].read()))
    except ClientError as e:
        print(f"読み込みエラー ({key}): {e}")
        return None

def read_csv_chunked_from_s3(s3_client, bucket, key, chunksize):
    """S3からCSVをチャンク単位で読み込み"""
    try:
        response = s3_client.get_object(Bucket=bucket, Key=key)
        body = response['Body'].read()
        return pd.read_csv(io.BytesIO(body), chunksize=chunksize)
    except ClientError as e:
        print(f"読み込みエラー ({key}): {e}")
        return None

def upload_to_s3(s3_client, bucket, key, data):
    """S3にファイルをアップロード"""
    try:
        s3_client.put_object(Bucket=bucket, Key=key, Body=data)
        return True
    except ClientError as e:
        print(f"アップロードエラー ({key}): {e}")
        return False

# S3クライアント作成
s3_client = get_s3_client(AWS_PROFILE)
print('S3クライアント作成完了')

S3クライアント作成完了


### 4. 比較処理関数

In [26]:
def get_file_config(filename):
    if filename in FILE_SETTINGS:
        return FILE_SETTINGS[filename]
    return {'key_columns': DEFAULT_KEY_COLUMNS, 'ignore_columns': DEFAULT_IGNORE_COLUMNS}

def compare_csv_normal(baseline_df, candidate_df, key_columns, ignore_columns):
    if len(key_columns) > 1:
        baseline_df = baseline_df.copy()
        candidate_df = candidate_df.copy()
        combined_key = '_'.join(key_columns)
        baseline_df[combined_key] = baseline_df[key_columns].astype(str).agg('_'.join, axis=1)
        candidate_df[combined_key] = candidate_df[key_columns].astype(str).agg('_'.join, axis=1)
        key_column = combined_key
    else:
        key_column = key_columns[0]
    
    baseline_indexed = baseline_df.set_index(key_column)
    candidate_indexed = candidate_df.set_index(key_column)
    compare_columns = [col for col in baseline_df.columns if col not in key_columns and col not in ignore_columns and col != key_column]
    diff_records = []
    
    for key in baseline_indexed.index.difference(candidate_indexed.index):
        diff_records.append({'key': key, 'diff_type': 'DELETED', 'column': None, 'baseline_value': None, 'candidate_value': None})
    for key in candidate_indexed.index.difference(baseline_indexed.index):
        diff_records.append({'key': key, 'diff_type': 'ADDED', 'column': None, 'baseline_value': None, 'candidate_value': None})
    
    for key in baseline_indexed.index.intersection(candidate_indexed.index):
        for col in compare_columns:
            bv, cv = baseline_indexed.loc[key, col], candidate_indexed.loc[key, col]
            if not (pd.isna(bv) and pd.isna(cv)) and bv != cv:
                diff_records.append({'key': key, 'diff_type': 'MODIFIED', 'column': col, 'baseline_value': bv, 'candidate_value': cv})
    return pd.DataFrame(diff_records)

def display_diff_by_id(diff_df, baseline_df, candidate_df, key_columns, ignore_columns):
    if len(diff_df) == 0:
        print('差分はありません')
        return
    if len(key_columns) > 1:
        baseline_df, candidate_df = baseline_df.copy(), candidate_df.copy()
        key_column = '_'.join(key_columns)
        baseline_df[key_column] = baseline_df[key_columns].astype(str).agg('_'.join, axis=1)
        candidate_df[key_column] = candidate_df[key_columns].astype(str).agg('_'.join, axis=1)
        key_display = '+'.join(key_columns)
    else:
        key_column, key_display = key_columns[0], key_columns[0]
    
    baseline_indexed = baseline_df.set_index(key_column)
    candidate_indexed = candidate_df.set_index(key_column)
    
    for key in sorted(diff_df['key'].unique()):
        diff_type = diff_df[diff_df['key'] == key]['diff_type'].iloc[0]
        print(f'\n{key_display}: {key}')
        print('=' * 40)
        
        if diff_type == 'DELETED':
            vals = [str(baseline_indexed.loc[key, c]) for c in baseline_indexed.columns if c not in ignore_columns and c not in key_columns]
            print(f'(前回の状態) {", ".join(vals)}')
            print('(今回の状態) ---')
        elif diff_type == 'ADDED':
            vals = [str(candidate_indexed.loc[key, c]) for c in candidate_indexed.columns if c not in ignore_columns and c not in key_columns]
            print('(前回の状態) ---')
            print(f'(今回の状態) {", ".join(vals)}')
        elif diff_type == 'MODIFIED':
            bvals = [str(baseline_indexed.loc[key, c]) for c in baseline_indexed.columns if c not in ignore_columns and c not in key_columns]
            cvals = [str(candidate_indexed.loc[key, c]) for c in candidate_indexed.columns if c not in ignore_columns and c not in key_columns]
            print(f'(前回の状態) {", ".join(bvals)}')
            print(f'(今回の状態) {", ".join(cvals)}')
            print(f'変更された項目: {", ".join(diff_df[diff_df["key"] == key]["column"].tolist())}')

def compare_csv_large(s3_client, bucket, key_a, key_b, key_columns, ignore_columns):
    size_a, size_b = get_s3_file_size(s3_client, bucket, key_a), get_s3_file_size(s3_client, bucket, key_b)
    print(f"ファイルサイズ: PrefixA={size_a:.1f}MB, PrefixB={size_b:.1f}MB, 合計={size_a+size_b:.1f}MB")
    print("通常処理を実行します...")
    baseline_df, candidate_df = read_csv_from_s3(s3_client, bucket, key_a), read_csv_from_s3(s3_client, bucket, key_b)
    if baseline_df is None or candidate_df is None:
        return None, None, None
    return compare_csv_normal(baseline_df, candidate_df, key_columns, ignore_columns), baseline_df, candidate_df

print('比較関数・詳細表示関数定義完了')

比較関数定義完了


### 5. ファイル一覧取得

In [27]:
# ファイル一覧取得
prefix_a_files = set(list_s3_csv_files(s3_client, S3_BUCKET, S3_PREFIX_A))
prefix_b_files = set(list_s3_csv_files(s3_client, S3_BUCKET, S3_PREFIX_B))
files_to_compare = list(prefix_a_files & prefix_b_files)

print(f'PrefixAのファイル: {sorted(prefix_a_files)}')
print(f'PrefixBのファイル: {sorted(prefix_b_files)}')
print(f'\n比較対象ファイル: {len(files_to_compare)}件')
print(f'ファイル: {sorted(files_to_compare)}')

PrefixAのファイル: ['sample.csv', 'tccontract_20251217.csv']
PrefixBのファイル: ['sample.csv', 'tccontract_20251217.csv']

比較対象ファイル: 2件
ファイル: ['sample.csv', 'tccontract_20251217.csv']


### 6. 比較実行

In [28]:
# 各ファイルを比較
results = {}

for filename in files_to_compare:
    print(f'\n{"="*60}')
    print(f'=== {filename} ===')
    print(f'{"="*60}')
    
    key_a = f"{S3_PREFIX_A.rstrip('/')}/{filename}"
    key_b = f"{S3_PREFIX_B.rstrip('/')}/{filename}"
    file_config = get_file_config(filename)
    
    diff_df, baseline_df, candidate_df = compare_csv_large(s3_client, S3_BUCKET, key_a, key_b, file_config['key_columns'], file_config['ignore_columns'])
    
    if diff_df is None:
        print('比較失敗')
        continue
    
    print(f'\n差分: {len(diff_df):,}件')
    
    if len(diff_df) > 0:
        summary = diff_df['diff_type'].value_counts()
        for diff_type, count in summary.items():
            print(f'  {diff_type}: {count:,}件')
        
        print(f'\n差分サンプル（最初の10件）:')
        display(diff_df.head(10))
        
        print(f'\n=== {filename} ID別詳細差分レポート ===\n')
        display_diff_by_id(diff_df, baseline_df, candidate_df, file_config['key_columns'], file_config['ignore_columns'])
        
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        output_key = f"{S3_OUTPUT_PREFIX.rstrip('/')}/{filename.replace('.csv', '')}_diff_{timestamp}.csv"
        csv_buffer = io.StringIO()
        diff_df.to_csv(csv_buffer, index=False, encoding='utf-8-sig')
        
        if upload_to_s3(s3_client, S3_BUCKET, output_key, csv_buffer.getvalue().encode('utf-8-sig')):
            print(f'\n出力: s3://{S3_BUCKET}/{output_key}')
        
        results[filename] = {'diff_count': len(diff_df), 'summary': summary.to_dict(), 'output_key': output_key}
    else:
        print('差分なし')
        results[filename] = {'diff_count': 0}
    
    gc.collect()

print(f'\n{"="*60}')
print('処理完了')
print(f'{"="*60}')


=== tccontract_20251217.csv ===
キー列: ['vin']
無視列: []
ファイルサイズ: PrefixA=1.2MB, PrefixB=1.2MB, 合計=2.5MB
通常処理を実行します...


詳細比較: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17405/17405 [00:02<00:00, 7134.76it/s]


差分: 4件
  MODIFIED: 2件
  DELETED: 1件
  ADDED: 1件

差分サンプル（最初の10件）:


,key,diff_type,column,baseline_value,candidate_value
0,PHY0620-0014890,DELETED,None,NaN,None
1,PHY0620-0013594,ADDED,None,NaN,None
2,PHY0620-0014884,MODIFIED,diffflag,111111111.0,111111110
3,PHY0620-0012225,MODIFIED,contractstartdatetime,NaN,test



出力: s3://k-ishibashi-test/output/tccontract_20251217_diff_20251222_184013.csv

=== sample.csv ===
キー列: ['vin']
無視列: []
ファイルサイズ: PrefixA=0.0MB, PrefixB=0.0MB, 合計=0.0MB
通常処理を実行します...


詳細比較: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 5787.91it/s]


差分: 8件
  MODIFIED: 6件
  DELETED: 1件
  ADDED: 1件

差分サンプル（最初の10件）:


,key,diff_type,column,baseline_value,candidate_value
0,4,DELETED,None,None,None
1,5,ADDED,None,None,None
2,1,MODIFIED,age,30,31
3,1,MODIFIED,salary,5000000,5200000
4,1,MODIFIED,updated_at,2024-01-01,2024-01-15
5,2,MODIFIED,updated_at,2024-01-01,2024-01-15
6,3,MODIFIED,department,人事,経理
7,3,MODIFIED,updated_at,2024-01-01,2024-01-15



出力: s3://k-ishibashi-test/output/sample_diff_20251222_184014.csv

処理完了


### 7. 結果サマリー

In [29]:
# 結果サマリー表示
print('\n=== 比較結果サマリー ===')
for filename, result in results.items():
    print(f'\n{filename}:')
    print(f'  差分件数: {result["diff_count"]:,}件')
    if 'summary' in result:
        for diff_type, count in result['summary'].items():
            print(f'    {diff_type}: {count:,}件')
    if 'output_key' in result:
        print(f'  出力: s3://{S3_BUCKET}/{result["output_key"]}')


=== 比較結果サマリー ===

tccontract_20251217.csv:
  差分件数: 4件
    MODIFIED: 2件
    DELETED: 1件
    ADDED: 1件
  出力: s3://k-ishibashi-test/output/tccontract_20251217_diff_20251222_184013.csv

sample.csv:
  差分件数: 8件
    MODIFIED: 6件
    DELETED: 1件
    ADDED: 1件
  出力: s3://k-ishibashi-test/output/sample_diff_20251222_184014.csv
